# Ancestral SNPs in the 1000 Genomes Project

## Introduction

In this notebook, we will extract the pre-defined fingerprinting (ancestral) SNPs from the vcf files of the 1000 Genomes Project. We will proceed through several steps to ensure we have a clean dataset:

1. Extracting relevant SNP positions.
2. Combining SNP data from various vcf files.
3. Removing structural variants.
4. Addressing duplicated chromosomal positions.
5. Validating our extraction.
6. Compressing and indexing the final vcf for efficient usage.

## Extracting the fingeprinting SNPs

In [1]:
# Let's begin by exploring and preparing our SNP data.
head data/fingerprinting_snps_biallelic.tsv | cut -f 1-8

chromosome	position	class	rsid	gene	alleles	ancestral_allele	variation_allele
1	1220751	snv	rs2887286	SDF4	T,C	T	C
1	2352457	snv	rs2840528	MORN1/LOC100129534	A,G	A	G
1	2622185	snv	rs3890745	MMEL1	T,C	T	C
1	3765267	snv	rs1181875	CCDC27	T,A,C	T	A,C
1	3826755	snv	rs6663840	CEP104	G,A,C	G	A,C
1	4304166	snv	rs693734		C,A,G,T	C	A,G,T
1	4436599	snv	rs1674877	LOC105376674	C,T	C	T
1	4657331	snv	rs1483198	AJAP1	A,G	A	G
1	4927364	snv	rs6666453		T,C,G	T	C,G


In [2]:
cat data/fingerprinting_snps_biallelic.tsv | cut -f 1,2 | sed '1d' | sed 's/^/chr/g' > working/fingerprinting_snps_biallelic_chr.txt
head working/fingerprinting_snps_biallelic_chr.txt

chr1	1220751
chr1	2352457
chr1	2622185
chr1	3765267
chr1	3826755
chr1	4304166
chr1	4436599
chr1	4657331
chr1	4927364
chr1	5028557


In [3]:
wc -l working/fingerprinting_snps_biallelic_chr.txt

8710 working/fingerprinting_snps_biallelic_chr.txt


In [4]:
# Create a combined fingerprinting vcf file
output="working/1kGP.fingerprinting.staging"
echo $output

working/1kGP.fingerprinting.staging


In [5]:
# Extract the header from the first vcf file
bcftools view -h `cat data/1kGP.vcfs.txt | head -1` > $output.vcf

The paths of the vcfs of the chromosomes of the 1000 Genomes Project are in the text file `1kGP.vcfs.txt`. The vcfs were downloaded from the [1000 Genomes 30x on GRCh38](https://www.internationalgenome.org/data-portal/data-collection/30x-grch38)'s [phased SNV/INDEL/SV calls of 3,202 samples](http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000G_2504_high_coverage/working/20220422_3202_phased_SNV_INDEL_SV/)

In [6]:
cat data/1kGP.vcfs.txt

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr1.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr2.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr3.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr4.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr5.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr6.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr7.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr8.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr9.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr10.filtered.SNV_INDEL_SV_phased_panel.vcf.gz
/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr11.filtered.SNV_INDEL_SV_phased_panel.vcf.

In [7]:
# Loop through each 1kGP chromosome in vcfs.txt and append non-header lines
while read -r vcf_file; do
    echo $vcf_file
    time bcftools view -H --regions-file working/fingerprinting_snps_biallelic_chr.txt $vcf_file >> $output.vcf
    echo
done < data/1kGP.vcfs.txt

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr1.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m6.766s
user	0m6.712s
sys	0m0.052s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr2.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m7.376s
user	0m7.279s
sys	0m0.096s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr3.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m5.698s
user	0m5.637s
sys	0m0.060s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr4.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m5.262s
user	0m5.209s
sys	0m0.052s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr5.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m5.462s
user	0m5.397s
sys	0m0.064s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr6.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m4.974s
user	0m4.925s
sys	0m0.048s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr7.filtered.SNV_INDEL_SV_phased_panel.vcf.gz

real	0m4.518s
user	0m4.469s
sys	0m0.048s

/data/1kGP/vcfs/1kGP_high_coverage_Illumina.chr8.filtered.SNV_

In [8]:
# Number of variants in the generated vcf
bcftools view -H $output.vcf | wc -l

9768


In [9]:
# Number of variants in the fingerprinting list
wc -l working/fingerprinting_snps_biallelic_chr.txt

8710 working/fingerprinting_snps_biallelic_chr.txt


In [10]:
# Take a glimpse at the generated vcf
bcftools view -H $output.vcf | tail | cut -f 1-5

chr22	48977019	22:48977019:A:G	A	G
chr22	49053806	22:49053806:T:G	T	G
chr22	49390502	22:49390502:G:A	G	A
chr22	49421090	HGSV_248270	A	<DEL>
chr22	49457563	22:49457563:G:A	G	A
chr22	49533849	22:49533849:T:C	T	C
chr22	50018612	22:50018612:C:T	C	T
chr22	50118978	22:50118978:A:G	A	G
chr22	50226183	22:50226183:A:G	A	G
chr22	50577409	22:50577409:T:C	T	C


## Remove Structural Variants (SV)
Keep only the variants with allele sizes = 1bp (e.g., remove chr22:49421090 [HGSV_248270]).

In [11]:
bcftools view -i 'strlen(REF) = 1 && strlen(ALT) = 1' $output.vcf  > $output"_without_sv".vcf

In [12]:
# Display the last few entries of the VCF file after removing structural variants to verify the changes
bcftools view -H $output"_without_sv".vcf | tail | cut -f 1-5

chr22	48783199	22:48783199:A:G	A	G
chr22	48977019	22:48977019:A:G	A	G
chr22	49053806	22:49053806:T:G	T	G
chr22	49390502	22:49390502:G:A	G	A
chr22	49457563	22:49457563:G:A	G	A
chr22	49533849	22:49533849:T:C	T	C
chr22	50018612	22:50018612:C:T	C	T
chr22	50118978	22:50118978:A:G	A	G
chr22	50226183	22:50226183:A:G	A	G
chr22	50577409	22:50577409:T:C	T	C


## Remove Duplicated Chromosomal Positions
This is most likely due to multiallelic variants in the 1kGP.

In [13]:
# Identify and save duplicated chromosomal positions to a file.
bcftools view -H $output"_without_sv".vcf | cut -f1,2 | sort | uniq -d > working/duplicated_positions.txt
cat working/duplicated_positions.txt

chr10	125555957
chr10	7384682
chr1	110189511
chr11	18595637
chr11	86633635
chr12	56894884
chr13	110364539
chr17	34702551
chr17	5015163
chr1	77361047
chr18	78847535
chr2	215946984
chr2	233852579
chr3	59184489
chr4	113640577
chr5	13847875
chr5	2697846


In [14]:
# Remove SNPs with duplicated chromosomal positions from the VCF file.
bcftools view -T ^working/duplicated_positions.txt -o $output"_without_sv_without_duplicates".vcf $output"_without_sv".vcf

In [15]:
cat working/fingerprinting_snps_biallelic_chr.txt | sed 's/\t/-/g' | sort -u > working/in.snps.txt
head working/in.snps.txt

chr10-100057800
chr10-100152307
chr10-100349445
chr10-100499949
chr10-100881934
chr10-10116693
chr10-101195800
chr10-101608897
chr10-101692888
chr10-101930813


In [16]:
# Count the number of unique SNPs in the transformed data.
wc -l working/in.snps.txt

8710 working/in.snps.txt


In [17]:
bcftools view -H $output"_without_sv_without_duplicates".vcf | cut -f 1,2 | sed 's/\t/-/g' | sort -u > working/out.snps.txt
head working/out.snps.txt

chr10-100057800
chr10-100152307
chr10-100349445
chr10-100499949
chr10-100881934
chr10-10116693
chr10-101195800
chr10-101608897
chr10-101692888
chr10-101930813


In [18]:
# Count the number of unique SNPs extracted from the processed vcf file.
wc -l working/out.snps.txt

8687 working/out.snps.txt


In [19]:
# Identify SNPs from the original list that didn't make it to the final vcf.
grep -v -f working/out.snps.txt working/in.snps.txt

chr10-125555957
chr10-7384682
chr1-110189511
chr11-18595637
chr11-86633635
chr12-56894884
chr13-110364539
chr1-33689901
chr1-62109755
chr17-34702551
chr17-5015163
chr1-77361047
chr18-78847535
chr2-162272314
chr2-215946984
chr2-233852579
chr3-59184489
chr4-113640577
chr5-13847875
chr5-2697846
chr7-81318201
chr8-71685451
chr9-18595550


In [20]:
# Compress the clean vcf
cat $output"_without_sv_without_duplicates".vcf | bgzip -c > data/1kGP.fingerprinting.vcf.gz

In [21]:
# Index the compressed clean vcf
tabix data/1kGP.fingerprinting.vcf.gz

In [22]:
# Check the final clean 1000GP vcf with the fingerprinting snps
bcftools view -H data/1kGP.fingerprinting.vcf.gz | tail | cut -f 1-20

chr22	48783199	22:48783199:A:G	A	G	.	.	AC=1954;AF=0.305122;CM=77.3726;AN=6404;AN_EAS=1170;AN_AMR=980;AN_EUR=1266;AN_AFR=1786;AN_SAS=1202;AN_EUR_unrel=1006;AN_EAS_unrel=1008;AN_AMR_unrel=694;AN_SAS_unrel=978;AN_AFR_unrel=1322;AF_EAS=0.157265;AF_AMR=0.276531;AF_EUR=0.414692;AF_AFR=0.407055;AF_SAS=0.205491;AF_EUR_unrel=0.411531;MAF_EUR_unrel=0.411531;AF_EAS_unrel=0.159722;MAF_EAS_unrel=0.159722;AF_AMR_unrel=0.285303;MAF_AMR_unrel=0.285303;AF_SAS_unrel=0.201431;MAF_SAS_unrel=0.201431;AF_AFR_unrel=0.405446;MAF_AFR_unrel=0.405446;AC_EAS=184;AC_AMR=271;AC_EUR=525;AC_AFR=727;AC_SAS=247;AC_EUR_unrel=414;AC_EAS_unrel=161;AC_AMR_unrel=198;AC_SAS_unrel=197;AC_AFR_unrel=536;AC_Het_EAS=164;AC_Het_AMR=179;AC_Het_EUR=309;AC_Het_AFR=405;AC_Het_SAS=193;AC_Het_EUR_unrel=246;AC_Het_EAS_unrel=143;AC_Het_AMR_unrel=122;AC_Het_SAS_unrel=159;AC_Het_AFR_unrel=298;AC_Het=1250;AC_Hom_EAS=20;AC_Hom_AMR=92;AC_Hom_EUR=216;AC_Hom_AFR=322;AC_Hom_SAS=54;AC_Hom_EUR_unrel=168;AC_Hom_EAS_unrel=18;AC_Hom_AMR_unrel=76;AC_Ho